In [18]:
import csv
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#Gọi các thư viện cần thiết

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")  # Chạy ẩn trình duyệt
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
service = Service("/usr/bin/chromedriver")
driver = webdriver.Chrome(service=service,options=chrome_options)

max_pages = 1
base_url = "https://alonhadat.com.vn/can-ban-nha"
data_list = []
#Cấu hình các thông tin cần thiết, rồi bắt đầu code lấy dưz liệu
for page in range(1, max_pages + 1):
    if page == 1:
        url = base_url + ".htm"
    else:
        url = base_url + f"/trang-{page}.htm"

    print(f"Đang thu thập dữ liệu từ: {url}")
    driver.get(url)
    time.sleep(5)  

    posts = driver.find_elements(By.CLASS_NAME, "content-item")

    if not posts:
        print("Không tìm thấy bài đăng nào, có thể đã hết trang.")
        break

    for post in posts:
        try:
                        
            diachi_element = post.find_element(By.CLASS_NAME, "ct_dis")
            diachi = diachi_element.text.strip()
            lists = diachi.split(", ")
            if "Đường" in lists[0]:
                lists[0] = lists[0].replace("Đường ", "")
            if "Phố" in lists[0]:
                lists[0] = lists[0].replace("Phố ", "")
            if "Phường" in lists[1]:
                lists[1] = lists[1].replace("Phường ", "")
            if "Xã" in lists[1]:
                lists[1] = lists[1].replace("Xã ", "")
            if "Quận" in lists[2]:
                lists[2] = lists[2].replace("Quận ", "")
            if "Huyện" in lists[2]:
                lists[2] = lists[2].replace("Huyện ", "")
            
            try:
                area_element = post.find_element(By.CLASS_NAME, "ct_dt")
                area = area_element.text.strip()
                area = area.replace("Diện tích: ", "")
                match = re.search(r"(\d+)\s*m", area)
                area = match.group(1) if match else None
            except:
                area = None
                
            try:
                kthuoc_element = post.find_element(By.CLASS_NAME, "ct_kt")
                kthuoc = kthuoc_element.text.strip()
                kthuoc = kthuoc.replace("KT: ", "")
                kthuoc = kthuoc.replace("m", "")
                kthuoc_list = kthuoc.split("x")
                cngang = kthuoc_list[0]
                cdai = kthuoc_list[1]
                if kthuoc == "---":
                    cngang = None
                    cdai = None
            except:
                cngang = None
                cdai = None
            try:
                duong_element = post.find_element(By.CLASS_NAME, "road-width")
                dorongduong = duong_element.text.strip()
                dorongduong = dorongduong.replace("m", "")
            except:
                dorongduong = None
                
            try:
                sotang_element = post.find_element(By.CLASS_NAME, "floors")
                sotang = sotang_element.text.strip()
                sotang = sotang.replace(" lầu", "")
            except:
                sotang = None
            
            try:
                sopngu_element = post.find_element(By.CLASS_NAME, "bedroom")
                sophongngu = sopngu_element.text.strip()
                sophongngu = sophongngu.replace(" phòng ngủ", "")
            except:
                sophongngu = None
            
            try:
                chodexe_element = post.find_element(By.CLASS_NAME, "parking")
                chodexe = chodexe_element.text.strip()
                chodexe = "Có"
            except:
                chodexe = None
            
            try:
                price_element = post.find_element(By.CLASS_NAME, "ct_price")
                price = price_element.text.strip()
                price = price.replace("Giá: ", "")
                price = price.replace(" tỷ", "")
                price = price.replace(" triệu", "")
            except:
                price = None
            
            data_list.append([lists[0], lists[1], lists[2], lists[3], area, cngang, cdai, dorongduong, sotang, sophongngu, chodexe, price])

        except Exception as e:
            print(f"Lỗi: {e}")
            continue

with open("Output.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Đường/Phố", "Phường/Xã", "Quận/Huyện", "Thành phố/Tỉnh", "Diện tích", "Chiều ngang", "Chiều dài", "Đường trước nhà", "Số tầng", "Số phòng ngủ", "Chỗ để xe", "Giá"])
    writer.writerows(data_list)

print("Dữ liệu đã được lưu vào Output.csv")

# Đóng trình duyệt
driver.quit()

Đang thu thập dữ liệu từ: https://alonhadat.com.vn/can-ban-nha.htm
Dữ liệu đã được lưu vào Output.csv
